# Requirements

In [99]:
# !pip install tenseal
# !pip install deepface


In [100]:
import tenseal as ts # pip install tenseal
from deepface import DeepFace #!pip install deepface
import base64
from deepface.commons import distance as dst

# Finding embeddings

We are going to find vector representations of facial images. This will be done in the client side.

In [101]:
img1_path = "img1.jpg"
img2_path = "img48.jpg"

In [102]:
img1_embedding = DeepFace.represent(img1_path, model_name = 'Facenet')
img2_embedding = DeepFace.represent(img2_path, model_name = 'Facenet')

# Commons

In [103]:
def write_data(file_name, data):

    if type(data) == bytes:
        #bytes to base64
        data = base64.b64encode(data)

    with open(file_name, 'wb') as f:
        f.write(data)

def read_data(file_name):
    with open(file_name, "rb") as f:
        data = f.read()

    #base64 to bytes
    return base64.b64decode(data)

# Initialization

We are going to generate secret - public key pair in this stage. This will be done in the client side.

In [104]:
context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree = 8192,
            coeff_mod_bit_sizes = [60, 40, 40, 60]
          )

context.generate_galois_keys()
context.global_scale = 2**40

In [105]:
secret_context = context.serialize(save_secret_key = True)
write_data("secret.txt", secret_context)

In [106]:
context.make_context_public() #drop the secret_key from the context
public_context = context.serialize()
write_data("public.txt", public_context)

In [107]:
del context, secret_context, public_context

# Encryption

We are going to apply homomorphic encryption to facial embeddings. This will be done in the client side.

Then, homomorphic encrypted facial embeddings will be stored in the cloud.

In [108]:
# Encryption

# Load secret key and context
context = ts.context_from(read_data("secret.txt"))

# Load facial embeddings
img1_embedding = DeepFace.represent(img1_path, model_name='Facenet')
img2_embedding = DeepFace.represent(img2_path, model_name='Facenet')

# Convert embeddings to flat lists
flat_img1_embedding = img1_embedding[0]['embedding']
flat_img2_embedding = img2_embedding[0]['embedding']

# Create CKKS vectors
enc_v1 = ts.ckks_vector(context, flat_img1_embedding)
enc_v2 = ts.ckks_vector(context, flat_img2_embedding)

# Serialize and save encrypted vectors
enc_v1_proto = enc_v1.serialize()
enc_v2_proto = enc_v2.serialize()

write_data("enc_v1.txt", enc_v1_proto)
write_data("enc_v2.txt", enc_v2_proto)

del context, enc_v1, enc_v2, enc_v1_proto, enc_v2_proto


# Calculations

Once homomorphic encrypted facial embeddings stored in the cloud, we are able to make calculations on encrypted data.

Notice that we just have public key here and we don't have secret key.

In [109]:
context = ts.context_from(read_data("public.txt"))

In [110]:
enc_v1_proto = read_data("enc_v1.txt")
enc_v2_proto = read_data("enc_v2.txt")

In [111]:
enc_v1 = ts.lazy_ckks_vector_from(enc_v1_proto)
enc_v1.link_context(context)

enc_v2 = ts.lazy_ckks_vector_from(enc_v2_proto)
enc_v2.link_context(context)

In [112]:
euclidean_squared = enc_v1 - enc_v2
euclidean_squared = euclidean_squared.dot(euclidean_squared)

In [113]:
write_data("euclidean_squared.txt", euclidean_squared.serialize())

In [115]:
del context, enc_v1_proto, enc_v2_proto, enc_v1, enc_v2, euclidean_squared

# Decryption

Once homomorphic encrypted euclidean squared value found in the cloud, we are going to retrieve it to the client side.

Client can decrypt it because we have the secret key in the client side.

In [116]:
context = ts.context_from(read_data("secret.txt"))

In [117]:
euclidean_squared_proto = read_data("euclidean_squared.txt")

In [118]:
euclidean_squared = ts.lazy_ckks_vector_from(euclidean_squared_proto)
euclidean_squared.link_context(context)

In [119]:
euclidean_squared_plain = euclidean_squared.decrypt()[0]

In [120]:
euclidean_squared_plain

180.60738817936164

In [121]:
if euclidean_squared_plain < 100:
    print("they are same person")
else:
    print("they are different persons")

they are different persons


# Validation

What if euclidean distance calculation is done in the client side always? Result should be same!

In [122]:
# Extract embeddings from dictionaries
img1_embedding_array = np.array(img1_embedding[0]['embedding'])
img2_embedding_array = np.array(img2_embedding[0]['embedding'])

# Calculate Euclidean distance
distance = dst.findEuclideanDistance(img1_embedding_array, img2_embedding_array)

# Output the Euclidean distance
print("Euclidean distance - traditional: ", distance * distance)
print("Euclidean distance - homomorphic: ", euclidean_squared_plain)


Euclidean distance - traditional:  180.60736389869209
Euclidean distance - homomorphic:  180.60738817936164
